# Azure AI Search এবং NVIDIA NIM এবং LlamaIndex ইন্টিগ্রেশন

এই নোটবুকে, আমরা NVIDIA-এর AI মডেল এবং LlamaIndex ব্যবহার করে একটি শক্তিশালী Retrieval-Augmented Generation (RAG) পাইপলাইন তৈরি করার পদ্ধতি দেখাব। আমরা NVIDIA-এর LLM এবং embeddings ব্যবহার করব, সেগুলোকে Azure AI Search-এর সাথে ভেক্টর স্টোর হিসেবে ইন্টিগ্রেট করব এবং RAG-এর মাধ্যমে সার্চের গুণমান এবং দক্ষতা উন্নত করব।

## সুবিধাসমূহ
- **স্কেলযোগ্যতা**: NVIDIA-এর বড় ভাষার মডেল এবং Azure AI Search ব্যবহার করে স্কেলযোগ্য এবং দক্ষ রিট্রিভাল।
- **খরচ সাশ্রয়ী**: কার্যকর ভেক্টর স্টোরেজ এবং হাইব্রিড সার্চ কৌশল ব্যবহার করে সার্চ এবং রিট্রিভাল অপ্টিমাইজ করুন।
- **উচ্চ পারফরম্যান্স**: শক্তিশালী LLM এবং ভেক্টরাইজড সার্চ একত্রিত করে দ্রুত এবং আরও সঠিক উত্তর প্রদান।
- **গুণমান**: প্রাসঙ্গিক রিট্রিভড ডকুমেন্টের মাধ্যমে LLM-এর উত্তরকে ভিত্তি দিয়ে উচ্চ সার্চ গুণমান বজায় রাখুন।

## পূর্বশর্ত
- 🐍 Python 3.9 বা তার বেশি
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key, NVIDIA-এর LLM এবং Embeddings-এ NVIDIA NIM মাইক্রোসার্ভিসের মাধ্যমে অ্যাক্সেসের জন্য

## কাভার করা বৈশিষ্ট্যসমূহ
- ✅ NVIDIA LLM ইন্টিগ্রেশন (আমরা ব্যবহার করব [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (আমরা ব্যবহার করব [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI Search-এর উন্নত রিট্রিভাল মোড
- ✅ LlamaIndex দিয়ে ডকুমেন্ট ইনডেক্সিং
- ✅ Azure AI Search এবং LlamaIndex-এর সাথে NVIDIA LLM ব্যবহার করে RAG

চলুন শুরু করি!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## ইনস্টলেশন এবং প্রয়োজনীয়তা
Python সংস্করণ >3.10 ব্যবহার করে একটি Python পরিবেশ তৈরি করুন।

## শুরু করা!


NVIDIA AI Foundation মডেল ব্যবহার করতে আপনার একটি `NVIDIA_API_KEY` প্রয়োজন:  
1) [NVIDIA](https://build.nvidia.com/explore/discover)-এ একটি বিনামূল্যের অ্যাকাউন্ট তৈরি করুন।  
2) আপনার পছন্দের মডেলে ক্লিক করুন।  
3) Input-এর অধীনে, Python ট্যাব নির্বাচন করুন এবং **Get API Key**-এ ক্লিক করুন, তারপর **Generate Key**-এ ক্লিক করুন।  
4) তৈরি করা কীটি কপি করুন এবং NVIDIA_API_KEY হিসেবে সংরক্ষণ করুন। এরপর, আপনি এন্ডপয়েন্টগুলোতে অ্যাক্সেস পাবেন।  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG উদাহরণ LLM এবং এমবেডিং ব্যবহার করে
### ১) LLM আরম্ভ করুন
`llama-index-llms-nvidia`, যা NVIDIA-এর LLM সংযোগকারী নামে পরিচিত, আপনাকে NVIDIA API ক্যাটালগে উপলব্ধ মডেলগুলোর সাথে সংযোগ স্থাপন এবং সেগুলো থেকে জেনারেট করতে সাহায্য করে। চ্যাট সম্পূর্ণ মডেলগুলোর তালিকা দেখতে এখানে যান: https://build.nvidia.com/search?term=Text-to-Text

এখানে আমরা **mixtral-8x7b-instruct-v0.1** ব্যবহার করব।


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) এম্বেডিং আরম্ভ করুন  
`llama-index-embeddings-nvidia`, যা NVIDIA এর এম্বেডিংস কানেক্টর নামেও পরিচিত, আপনাকে NVIDIA API ক্যাটালগে উপলব্ধ মডেলগুলোর সাথে সংযোগ স্থাপন এবং সেগুলো থেকে জেনারেট করতে সাহায্য করে। আমরা এম্বেডিং মডেল হিসেবে `nvidia/nv-embedqa-e5-v5` নির্বাচন করেছি। টেক্সট এম্বেডিং মডেলগুলোর তালিকা দেখতে এখানে যান: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### ৩) একটি Azure AI Search Vector Store তৈরি করুন


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### ৫) আপনার ডেটার উপর প্রশ্ন করার জন্য একটি কুইরি ইঞ্জিন তৈরি করুন

এখানে Azure AI Search-এ খাঁটি ভেক্টর অনুসন্ধান ব্যবহার করে এবং আমাদের LLM (Phi-3.5-MOE)-এ উত্তর ভিত্তি করে একটি কুইরি দেওয়া হয়েছে।


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

এখানে Azure AI Search-এ হাইব্রিড সার্চ ব্যবহার করে একটি প্রশ্ন।


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### ভেক্টর অনুসন্ধান বিশ্লেষণ
LLM-এর উত্তর সঠিকভাবে রাশিয়ার শেয়ার বাজার সম্পর্কিত মূল অর্থনৈতিক প্রভাবগুলি তুলে ধরেছে যা উৎস পাঠ্যে উল্লেখ করা হয়েছে। বিশেষভাবে, এটি উল্লেখ করেছে যে রাশিয়ার শেয়ার বাজারে উল্লেখযোগ্য পতন ঘটেছে, যার ফলে বাজার তার ৪০% মূল্য হারিয়েছে এবং চলমান পরিস্থিতির কারণে লেনদেন স্থগিত করা হয়েছে। এই উত্তর উৎসে প্রদত্ত তথ্যের সাথে ভালোভাবে সামঞ্জস্যপূর্ণ, যা নির্দেশ করে যে LLM সঠিকভাবে প্রাসঙ্গিক বিবরণ চিহ্নিত এবং সংক্ষেপ করেছে, যা রাশিয়ার কার্যকলাপ এবং আরোপিত নিষেধাজ্ঞার ফলস্বরূপ শেয়ার বাজারের প্রভাবকে তুলে ধরে।

#### উৎস নোডের মন্তব্য
উৎস নোডগুলি আন্তর্জাতিক নিষেধাজ্ঞার কারণে রাশিয়া যে অর্থনৈতিক প্রভাবগুলির মুখোমুখি হয়েছে তার একটি বিস্তারিত বিবরণ প্রদান করে। পাঠ্যটি তুলে ধরে যে রাশিয়ার শেয়ার বাজার তার ৪০% মূল্য হারিয়েছে এবং লেনদেন স্থগিত করা হয়েছে। এছাড়াও, এটি অন্যান্য অর্থনৈতিক প্রভাবগুলির উল্লেখ করে, যেমন রুবলের অবমূল্যায়ন এবং রাশিয়ার অর্থনীতির বৃহত্তর বিচ্ছিন্নতা। LLM-এর উত্তর এই নোডগুলির থেকে গুরুত্বপূর্ণ বিষয়গুলি দক্ষতার সাথে সংক্ষেপ করেছে, বিশেষভাবে শেয়ার বাজারের প্রভাবের উপর মনোযোগ কেন্দ্রীভূত করেছে, যা প্রশ্নে অনুরোধ করা হয়েছিল।


এখন, চলুন এমন একটি প্রশ্নের দিকে নজর দিই যেখানে Hybrid Search একটি সুসংগঠিত উত্তর প্রদান করতে পারে না:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### হাইব্রিড সার্চ: এলএলএম প্রতিক্রিয়া বিশ্লেষণ
হাইব্রিড সার্চ উদাহরণে এলএলএম-এর প্রতিক্রিয়া নির্দেশ করে যে প্রদত্ত প্রসঙ্গে রাশিয়ার ইউক্রেন আক্রমণের সঠিক তারিখ উল্লেখ করা হয়নি। এই প্রতিক্রিয়া ইঙ্গিত দেয় যে এলএলএম উৎস ডকুমেন্টে থাকা তথ্য ব্যবহার করছে, তবে পাঠ্যে সুনির্দিষ্ট বিবরণের অভাবকে স্বীকার করছে।

প্রতিক্রিয়া সঠিকভাবে চিহ্নিত করেছে যে প্রসঙ্গে রাশিয়ার আগ্রাসনের সাথে সম্পর্কিত ঘটনাগুলি উল্লেখ করা হয়েছে, কিন্তু নির্দিষ্ট আক্রমণের তারিখ উল্লেখ করা হয়নি। এটি এলএলএম-এর তথ্য বোঝার ক্ষমতা এবং বিষয়বস্তুর ফাঁকগুলি চিহ্নিত করার দক্ষতা প্রদর্শন করে। এলএলএম কার্যকরভাবে ব্যবহারকারীকে সঠিক তারিখের জন্য বাহ্যিক উৎস বা ঐতিহাসিক রেকর্ড খুঁজতে উৎসাহিত করে, যখন তথ্য অসম্পূর্ণ থাকে তখন একটি সতর্ক মনোভাব প্রদর্শন করে।

### উৎস নোড বিশ্লেষণ
হাইব্রিড সার্চ উদাহরণে উৎস নোডগুলি একটি বক্তৃতার অংশ অন্তর্ভুক্ত করে যেখানে ইউক্রেনে রাশিয়ার কার্যকলাপের প্রতি যুক্তরাষ্ট্রের প্রতিক্রিয়া আলোচনা করা হয়েছে। এই নোডগুলি বিস্তৃত ভূরাজনৈতিক প্রভাব এবং আক্রমণের প্রতিক্রিয়ায় যুক্তরাষ্ট্র ও তার মিত্রদের নেওয়া পদক্ষেপগুলিকে গুরুত্ব দেয়, তবে নির্দিষ্ট আক্রমণের তারিখ উল্লেখ করে না। এটি এলএলএম-এর প্রতিক্রিয়ার সাথে সামঞ্জস্যপূর্ণ, যা সঠিকভাবে চিহ্নিত করেছে যে প্রসঙ্গে সুনির্দিষ্ট তারিখের তথ্যের অভাব রয়েছে।


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### হাইব্রিড w/রির‍্যাঙ্কিং: LLM প্রতিক্রিয়া বিশ্লেষণ
হাইব্রিড w/রির‍্যাঙ্কিং উদাহরণে, LLM প্রতিক্রিয়া অতিরিক্ত প্রসঙ্গ প্রদান করে উল্লেখ করে যে ঘটনাটি বক্তৃতা দেওয়ার ছয় দিন আগে ঘটেছিল। এটি নির্দেশ করে যে LLM বক্তৃতার সময়ের উপর ভিত্তি করে আক্রমণের তারিখ অনুমান করতে সক্ষম, যদিও সঠিকতার জন্য বক্তৃতার সঠিক তারিখ জানা এখনও প্রয়োজন।

এই প্রতিক্রিয়া প্রসঙ্গ সূত্র ব্যবহার করে আরও তথ্যপূর্ণ উত্তর দেওয়ার উন্নত ক্ষমতা প্রদর্শন করে। এটি রির‍্যাঙ্কিংয়ের সুবিধা তুলে ধরে, যেখানে LLM আরও প্রাসঙ্গিক তথ্য অ্যাক্সেস করতে এবং অগ্রাধিকার দিতে পারে, যা কাঙ্ক্ষিত বিশদ (যেমন, আক্রমণের তারিখ) এর কাছাকাছি একটি অনুমান প্রদান করে।

### সোর্স নোড বিশ্লেষণ
এই উদাহরণে সোর্স নোডগুলোতে রাশিয়ার আক্রমণের সময়ের উল্লেখ রয়েছে, বিশেষভাবে উল্লেখ করা হয়েছে যে এটি বক্তৃতার ছয় দিন আগে ঘটেছিল। যদিও সঠিক তারিখ এখনও স্পষ্টভাবে উল্লেখ করা হয়নি, নোডগুলো এমন সময়গত প্রসঙ্গ প্রদান করে যা LLM-কে আরও সূক্ষ্ম প্রতিক্রিয়া দিতে সাহায্য করে। এই বিশদ অন্তর্ভুক্তি দেখায় যে রির‍্যাঙ্কিং কীভাবে প্রদত্ত প্রসঙ্গ থেকে তথ্য বের করার এবং অনুমান করার LLM-এর ক্ষমতা উন্নত করতে পারে, যার ফলে আরও সঠিক এবং তথ্যপূর্ণ প্রতিক্রিয়া পাওয়া যায়।


**নোট:**
এই নোটবুকে আমরা NVIDIA API ক্যাটালগ থেকে NVIDIA NIM মাইক্রোসার্ভিস ব্যবহার করেছি।  
উপরের API গুলো, `NVIDIA (llms)`, `NVIDIAEmbedding`, এবং [Azure AI Search Semantic Hybrid Retrieval (বিল্ট-ইন রির‍্যাঙ্কিং)](https://learn.microsoft.com/azure/search/semantic-search-overview)।  
উল্লেখ্য, উপরোক্ত API গুলো স্ব-হোস্টেড মাইক্রোসার্ভিসকেও সমর্থন করতে পারে।  

**উদাহরণ:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
